# **Sistema Predictivo para Diagnosticar la Enfermedad de Parkinson Basado en Pruebas del Habla.**

Juan Pablo Gómez Restrepo,  juan.gomez125@udea.edu, Estudiante de ingeniería



Sebastian Gómez Molinasebastian, gomez19@udea.edu.co,  Estudiante de ingeniería

## **Seleccion y Extraccion de caracteristicas**

In [ ]:
!pip install mlxtend

In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow

In [ ]:
!pip install joblib

## IMPORTAR LIBRERIAS

In [ ]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import time
from scipy.spatial import distance
from scipy import stats
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.metrics import accuracy
import operator
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import GaussianNB
from mlxtend.preprocessing import standardize
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report
import sys
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
from sklearn import svm
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier

### **Cargar Base De Datos Y Conjunto De Muestras De Entrenamiento**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('train_data.txt', header=None)
test = pd.read_csv('test_data.txt', header=None)

In [ ]:
#Conjunto de muestras de entrenamiento
Xtrain = train.iloc[0:1040,1:26].to_numpy()
Ytrain = train.iloc[0:1040,28].to_numpy()

Xtest = test.iloc[0:168,1:26].to_numpy()
Ytest = test.iloc[0:168,27].to_numpy()

In [ ]:
Xtrain, Ytrain = shuffle(Xtrain, Ytrain)

### ***Seleccion de caracteristicas secuencial ascendente y descendente***

In [ ]:
def select_features(modelo, n_features, fwd, fltg):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0,
           n_jobs=4)#para desbaratar equipos
    
    return sfs

### ***Analisis Driscriminante Cuadratico***

In [ ]:
def error(Y_lest, Y):
    error = 0
    for ye, y in zip(Y_lest, Y):
        if ye != y:
            error += 1
    error = error/np.size(Y)
    return error

tiempo_i = time.time()

#Estandarizamos la base de datos
# Use Max - Min
scaler = MinMaxScaler()
scaler.fit(Xtrain)
X = scaler.transform(Xtrain)

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]   
    
    #Funcion d gaussiana de naive bayes
    gFN = GaussianNB()
    
    sf = select_features(gFN,10,False,True)
    
    sf = sf.fit(X_train,y_train)
    
    Errores[j] = 1-sf.k_score_
    j+=1
    
print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print ('\nTiempo total de ejecución: ' + str(time.time()-tiempo_i) + ' segundos.')


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    4.6s finished
Features: 24/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 23/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.4s finished
Features: 22/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    4.2s finished


Error de validación: 0.3731837606837607 +/- 0.011645299145299156

Tiempo total de ejecución: 121.74971771240234 segundos.


In [ ]:
y_estdg

array([1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int64)

In [ ]:
y_test

array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0], dtype=int64)

<table>
<tr>
<td>Error de validacion</td>
<td>std</td>
<td>tiempo de ejecucion</td>
</tr>
<tr>
<td>0.24404761904761904 </td>
<td>50</td>
<td>50</td>
</tr>
<tr>
<td>0.19642857142857142 </td>
<td>70</td>
<td>50</td>
</tr>
<tr>
<td>0.16666666666666666 </td>
<td>90</td>
<td>50</td>
</tr>
<tr>
<td>0.14285714285714285 </td>
<td>100</td>
<td>50</td>
</tr>
<tr>
<td>0.06547619047619048 </td>
<td>120</td>
<td>50</td>
</tr>
</table>

### ***Ventana de Parzen***

In [ ]:
def dist_euclidiana(x1, x2):
    
    dist = distance.euclidean(x1, x2)
    
    return dist

In [ ]:
# Error cuadrático medio para los problemas de regresión
def ECM(Y_est,Y):
    N = np.size(Y)
    Y = 100*Y    #Analice la variable de salida de la base de datos y observe porque se multiplica por 100
    Y_est = 100*np.asarray(Y_est)
    ecm = sum((Y_est - Y)**2)/(2*N)
    
    return ecm

#Para calcular el error en los problemas de clasificación
def error(Y_lest, Y):
    error = 0
    for ye, y in zip(Y_lest, Y):
        if ye != y:
            error += 1
    
    error = error/np.size(Y)
    
    return error

tiempo_i = time.time()


for n in range(1,10):
    for train_index, test_index in kf.split(X):

        X_train, X_test = Xtrain[train_index], Xtrain[test_index]
        y_train, y_test = Ytrain[train_index], Ytrain[test_index]   

        #Train Model and Predict  
        knn = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
        y_estknn = knn.predict(X_test)
        
        Errores[j] = error(y_estknn, y_test)
        

    

In [ ]:
#Para calcular el costo computacional
tiempo_i = time.time()
#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = Xtrain[train_index], Xtrain[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]  
   
    sf = select_features(knn,10,False,True)

    sf = sf.fit(X_train,y_train)
    
    Errores[j] = 1-sf.k_score_
    j+=1


print("\nCaracteristicas: "+ str(sf.k_feature_idx_))
print("\nError de validación aplicando SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print("\nEficiencia en validación aplicando SFS: " + str(sf.k_score_*100) + "%" )
print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")
print("\nCaracteristicas: "+ str(sf.k_feature_idx_))
print("\n")
print(classification_report(y_test,y_estknn,digits=2))
print("\nmatriz de confusión: \n" )
print(confusion_matrix(y_test, y_estknn, labels=[3,4,5,6,7,8,9]))
print("\n\n")

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    3.5s finished
Features: 24/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 23/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  23 out of  23 | elapsed:    0.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.0s finished
Features: 22/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    3.6s finished


Caracteristicas: (4, 8, 9, 10, 14, 16, 17, 18, 19, 20)

Error de validación aplicando SFS: 0.25427350427350426 +/- 0.0057731863615838675

Eficiencia en validación aplicando SFS: 75.0%


Tiempo total de ejecución: 182.44626259803772 segundos.

Caracteristicas: (4, 8, 9, 10, 14, 16, 17, 18, 19, 20)


              precision    recall  f1-score   support

           0       0.60      0.52      0.56        52
           1       0.58      0.65      0.61        52

    accuracy                           0.59       104
   macro avg       0.59      0.59      0.58       104
weighted avg       0.59      0.59      0.58       104


matriz de confusión: 



ValueError: At least one label specified must be in y_true

In [ ]:
KNN(Xtrain,Ytrain,Xtest,Ytest,120)

Modelo KNN para clasificación

El Error de clasificación del modelo con 120 vecinos es = 0.06547619047619048


Tiempo total de ejecución: 17.912153482437134 segundos.


<table>
<tr>
<td>Error de clasificacion</td>
<td>Numero de vecinos</td>
</tr>
<tr>
<td>0.24404761904761904 </td>
<td>50</td>
</tr>
<tr>
<td>0.19642857142857142 </td>
<td>70</td>
</tr>
<tr>
<td>0.16666666666666666 </td>
<td>90</td>
</tr>
<tr>
<td>0.14285714285714285 </td>
<td>100</td>
</tr>
<tr>
<td>0.06547619047619048 </td>
<td>120</td>
</tr>
</table>

### ***Gradiente Boosting Tree***

In [ ]:
def metrics(y_test, y_pred):
    y_test = np.asarray(y_test)
    y_pred = np.asarray(y_pred)


    TP,TN,FP,FN = 0,0,0,0
    for t,p in zip(y_test,y_pred):
        if t==1:
            if p>=0.5: #yp==1:
                TP+=1
            else:
                FN+=1
        else:
            if p>=0.5: #yp==1:
                FP+=1
            else:
                TN+=1

    acc = (TP+TN)/(TP+FP+FN+TN)

    return acc


In [ ]:
#Para calcular el costo computacional
tiempo_i = time.time()

#Complete el código para crear la red neuronal artificial.
#Para la configuración de la red observe la tabla de más abajo.
#Recuerde que debe usar la función identidad en la capa de salida (linear).
#Para la capa oculta puede use relu.

model = Sequential()
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(Xtrain, Ytrain):
    
    X_train, X_test = Xtrain[train_index], Xtrain[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]  

    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    sf = select_features(model,10,False,True)
    
    sf = sf.fit(X_train,y_train)
    

    Errores[j] = 1-sf.k_score_
    j+=1
    
print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x000001EBDB3F9A30>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

<table>
<tr>
<td>Error de validacion</td>
<td>std</td>
<td>tiempo de ejecucion</td>
</tr>
<tr>
<td>0.24404761904761904 </td>
<td>50</td>
<td>50</td>
</tr>
<tr>
<td>0.19642857142857142 </td>
<td>70</td>
<td>50</td>
</tr>
<tr>
<td>0.16666666666666666 </td>
<td>90</td>
<td>50</td>
</tr>
<tr>
<td>0.14285714285714285 </td>
<td>100</td>
<td>50</td>
</tr>
<tr>
<td>0.06547619047619048 </td>
<td>120</td>
<td>50</td>
</tr>
</table>

### ***Redes de Neuronalas Artificiales***

In [ ]:
def class_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):
        if y_e != y_r:
            err += 1

    return "accuracy: " + str(1-(err/np.size(y_est)))

In [ ]:
tiempo_i = time.time()

kf = KFold(n_splits=10)

Errores = np.zeros(10)

for B in range(1,10,2):
      
    Errores = np.zeros(10)
    j = 0
    #Aplicando la metodología de validación
      
    for train_index, test_index in kf.split(Xtrain):
          
        #Complete el código para tomar las muestras de entrenamiento y validación usando
        #train_index y test_index 

        X_train, X_test = Xtrain[train_index], Xtrain[test_index]
        y_train, y_test = Ytrain[train_index], Ytrain[test_index]
          
        #Complete el código aquí creado el modelo Random Forest 
        #ara clasificar y entrenandolo solo con las muestras de entrenamiento 
          
        RF = RandomForestClassifier(n_estimators=B)
        
        sf = select_features(RF,10,False,True)
    
        sf = sf.fit(X_train,y_train) #Complete el código aquí       
          
    
        Errores[j] = 1-sf.k_score_
        j += 1

    print('\nResultado de clasificación con ', B, 'árboles:')
    print ("accuracy: " + str(1-np.mean(Errores)))
    print ('\nTiempo total de ejecución: ' + str(time.time()-tiempo_i) + ' segundos.')


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    0.5s finished
Features: 24/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 23/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  16 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s

STOPPING EARLY DUE TO KEYBOARD INTERRUPT...[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25


Resultado de clasificación con  1 árboles:
accuracy: 0.8685897435897436

Tiempo total de ejecución: 112.47668290138245 segundos.


[Parallel(n_jobs=4)]: Done  18 out of  25 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    0.2s finished
Features: 24/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 23/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  23 out of  23 | elapsed:    1.9s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.4s finished
Features: 22/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    3.4s fin


Resultado de clasificación con  3 árboles:
accuracy: 0.9402777777777778

Tiempo total de ejecución: 252.6527886390686 segundos.


[Parallel(n_jobs=4)]: Done  18 out of  25 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    0.2s finished
Features: 24/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 23/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  23 out of  23 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.1s finished
Features: 22/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    3.4s fin


Resultado de clasificación con  5 árboles:
accuracy: 0.9711538461538461

Tiempo total de ejecución: 418.11429500579834 segundos.


[Parallel(n_jobs=4)]: Done  18 out of  25 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    0.5s finished
Features: 24/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 23/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  16 out of  23 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=4)]: Done  23 out of  23 | elapsed:    0.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.7s finished
Features: 22/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 conc


Resultado de clasificación con  7 árboles:
accuracy: 0.9821581196581196

Tiempo total de ejecución: 628.5181894302368 segundos.


[Parallel(n_jobs=4)]: Done  18 out of  25 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    0.6s finished
Features: 24/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 23/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  23 out of  23 | elapsed:    0.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.3s finished
Features: 22/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    3.9s fin


Resultado de clasificación con  9 árboles:
accuracy: 0.9917735042735043

Tiempo total de ejecución: 860.1650438308716 segundos.


<table>
<tr>
<td>Numero de árboles</td>
<td>accuaracy</td>
</tr>
<tr>
<td>0.4538461538461538 </td>
<td>1</td>
</tr>
<tr>
<td>0.4721153846153846 </td>
<td>3</td>
</tr>
<tr>
<td>0.4375 </td>
<td>5</td>
</tr>
<tr>
<td>0.4375000000000001 </td>
<td>7</td>
</tr>
<tr>
<td>0.40865384615384615 </td>
<td>9</td>
</tr>
</table>

### ***Maquinas de Soporte Vectorial***

In [ ]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

C_list = [100]
gamma_list = [0.0001]

for c in C_list:
    for g in gamma_list:

#Para calcular el costo computacional

        tiempo_i = time.time()

#Complete el código para crear el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
#Consulte como lo puede tratar con la SVM.
#Use un kernel rbf con una malla de valores así: C en {0.1,1,10,100} y gamma en {0.0001,0.01, 0.1}

        clf = svm.SVC(C = c, gamma = g, kernel='rbf', decision_function_shape = 'ovo')

#Implemetamos la metodología de validación cross validation con 10 folds

        Errores = np.ones(10)
        j = 0
        kf = KFold(n_splits=10)

        for train_index, test_index in kf.split(X):

            X_train, X_test = Xtrain[train_index], Xtrain[test_index]
            y_train, y_test = Ytrain[train_index], Ytrain[test_index]  

            
            sf = select_features(clf,10,False,True)
            
            sf = sf.fit(X_train,y_train)
        

            Errores[j] = 1-sf.k_score_
            j+=1

        print("\nError de validación aplicando SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

        print("\nEficiencia en validación aplicando SFS: " + str(sf.k_score_*100) + "%" )

        print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    1.6s finished
Features: 24/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
Features: 23/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  23 out of  23 | elapsed:    1.9s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.7s finished
Features: 22/10[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    4.7s finished


Error de validación aplicando SFS: 0.12104700854700856 +/- 0.0030612283721996474

Eficiencia en validación aplicando SFS: 87.71367521367522%


Tiempo total de ejecución: 355.70159578323364 segundos.
